In [1]:
import numpy
import cvxpy
import itertools

# SDP Problem

Using `cvxpy` to solve a semidefinite programing (SDP) problem.
$$\begin{split}&\mathrm{min}\;b^\intercal x\\
&\mathrm{s.t.}\;A_0+\sum_{i}x_i A_i\succeq 0\end{split}$$
The constraint matrices can be complex Hermitian matrices. 

In [7]:
import numpy
import cvxpy

b = numpy.array([1.])
As = numpy.array([[[1.,0.],[0.,1.]], [[0.,-1.j],[1.j,0.]]])

n = b.shape[0]
x = cvxpy.Variable(n)

objective = x @ b
A = As[0]
for i in range(n):
    A = A  As[i+1] * x[i]
constraints = [A >> 0]
print(A)

problem = cvxpy.Problem(cvxpy.Minimize(objective), constraints)
problem.solve()

print("The optimal value is", problem.value)
print("A solution x is\n", x.value)

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]] + [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]] * Promote(var104[0], (2, 2))
The optimal value is -0.9999999703672325
A solution x is
 [-0.99999997]


In [5]:
numpy.__version__, cvxpy.__version__

('1.17.2', '1.0.25')

pack into a function.

In [26]:
SDP(b, As)

(-0.9999999703672325, array([-0.99999997]))

## Operator Array

In [33]:
%run qboot

In [38]:
s = OperatorSpace([maj(i) for i in range(4)])
s

OperatorSpace([χ0, χ1, χ2, χ3])

In [40]:
s.reconstruct(s.represent(maj(2)))

χ2

In [39]:
s.reconstruct(s.represent([maj(2),maj(3)]))

Operators([χ2, χ3])

In [25]:
a = Operators([maj(i) for i in range(4)])
a

Operators([χ0, χ1, χ2, χ3])

In [26]:
0.3 * a, a * 0.5

(Operators([0.30 χ0, 0.30 χ1, 0.30 χ2, 0.30 χ3]),
 Operators([0.50 χ0, 0.50 χ1, 0.50 χ2, 0.50 χ3]))

The cummutative multiplication of operators is defined to be the operator inner product $A * B = \mathrm{Tr}(A B)$, this is the only way to define a meaningful cummutative multiplication for generally non-commutative operators.

In [22]:
maj() * maj(1), maj(1) * maj(1)

(0, 1)

In [65]:
maj(1) * a, a * maj(1)

(array([0, 1, 0, 0], dtype=object), array([0, 1, 0, 0], dtype=object))

In [27]:
a * (0.3*maj(1)+0.2*maj(2)) # use to perform operator representation

array([0. , 0.3, 0.2, 0. ])

In [28]:
a * a

array([1., 1., 1., 1.])

In [29]:
(a * a).astype(numpy.float), (a * a).astype(numpy.complex)

(array([1., 1., 1., 1.]), array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j]))

In [30]:
x = numpy.arange(4)
x

array([0, 1, 2, 3])

In [31]:
x * a, a * x

(Operators([0, χ1, 2 χ2, 3 χ3]), Operators([0, χ1, 2 χ2, 3 χ3]))

In [32]:
x.dot(a), a.dot(x) # use for operator reconstruction

(χ1 +2 χ2 +3 χ3, χ1 +2 χ2 +3 χ3)

In [103]:
b = numpy.array([maj(i,i+1).hermitianize() for i in range(4)])
b

array([i χ0 χ1, i χ1 χ2, i χ2 χ3, i χ3 χ4], dtype=object)

In [104]:
b.conj()

array([i χ0 χ1, i χ1 χ2, i χ2 χ3, i χ3 χ4], dtype=object)

In [105]:
H = sum(1j*maj(i,i+1) for i in range(4))
H

i χ0 χ1 +i χ1 χ2 +i χ2 χ3 +i χ3 χ4

In [106]:
[H.commutate(op) for op in a]

[-2i χ1, 2i χ0 -2i χ2, 2i χ1 -2i χ3, -2i χ4 +2i χ2]

In [107]:
mul_table = numpy.array([[op1 @ op2 for op1 in a] for op2 in a])
mul_table

array([[I, - χ0 χ1, - χ0 χ2, - χ0 χ3],
       [χ0 χ1, I, - χ1 χ2, - χ1 χ3],
       [χ0 χ2, χ1 χ2, I, - χ2 χ3],
       [χ0 χ3, χ1 χ3, χ2 χ3, I]], dtype=object)

In [111]:
b.reshape(-1,1,1).conjugate() * mul_table.reshape(1,4,4)

array([[[0, 1j, 0, 0],
        [-1j, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 1j, 0],
        [0, -1j, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1j],
        [0, 0, -1j, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=object)

In [132]:
c = numpy.array([op.hermitianize() for op in mul_table.flat])
c

array([I, -i χ0 χ1, -i χ0 χ2, -i χ0 χ3, i χ0 χ1, I, -i χ1 χ2, -i χ1 χ3,
       i χ0 χ2, i χ1 χ2, I, -i χ2 χ3, i χ0 χ3, i χ1 χ3, i χ2 χ3, I],
      dtype=object)

In [136]:
d = numpy.hstack([a,c])
d

array([χ0, χ1, χ2, χ3, I, -i χ0 χ1, -i χ0 χ2, -i χ0 χ3, i χ0 χ1, I,
       -i χ1 χ2, -i χ1 χ3, i χ0 χ2, i χ1 χ2, I, -i χ2 χ3, i χ0 χ3,
       i χ1 χ3, i χ2 χ3, I], dtype=object)

In [151]:
gram = numpy.outer(d,d).astype(numpy.complex).real
w, v = numpy.linalg.eigh(gram)

In [157]:
d.dot(v[:,w > 1.e-10].round(10))

array([χ0, χ1, χ3, χ2, 1.41i χ1 χ2 -0.04i χ1 χ3, -1.41i χ2 χ3,
       -i χ0 χ1 +i χ0 χ2, 1.41i χ0 χ3, 0.04i χ1 χ2 +1.41i χ1 χ3,
       i χ0 χ1 +i χ0 χ2, 2 I], dtype=object)

In [73]:
%run qboot

In [74]:
ops = OperatorSpace(MajoranaOperator, numpy.array([maj()]+[maj(i) for i in range(4)]))
ops

[I,
 χ0,
 χ1,
 χ2,
 χ3]

In [47]:
x = ops.represent(numpy.array([[maj(0),maj(1)],[maj(2),maj(3)]]))
x

array([[[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]]])

In [48]:
ops.reconstruct(x)

array([[χ0, χ1],
       [χ2, χ3]], dtype=object)

In [49]:
x = ops.represent(numpy.array([[maj(0),maj(1)],[maj(2),maj(3)]]), axis=0)
x

array([[[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]],

       [[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]],

       [[0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j],
        [1.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]])

In [51]:
ops.reconstruct(x, axis=0)

array([[χ0, χ1],
       [χ2, χ3]], dtype=object)

In [75]:
basis = numpy.random.randn(10,5) @ ops.basis
basis

array([-1.67 I +0.16 χ0 +1.17 χ1 -1.41 χ2 -0.18 χ3,
       -0.15 I +0.63 χ0 -0.95 χ1 -1.48 χ2 -1.93 χ3,
       0.76 I -1.13 χ0 -1.21 χ1 +0.39 χ2 -0.30 χ3,
       0.41 I -1.38 χ0 -0.75 χ1 -0.30 χ2 +1.18 χ3,
       -0.17 I +1.04 χ0 +0.86 χ1 -0.89 χ2 -1.34 χ3,
       -0.70 I +0.01 χ0 -2.07 χ1 +2.26 χ2 -0.67 χ3,
       -0.41 I +1.23 χ0 +1.19 χ1 +1.16 χ2 -1.71 χ3,
       1.50 I -0.36 χ0 +0.36 χ1 +0.87 χ2 +0.88 χ3,
       0.43 I -0.42 χ0 +0.17 χ1 +0.58 χ2 -1.33 χ3,
       0.20 I -0.57 χ0 +0.09 χ1 -0.13 χ2 +1.06 χ3], dtype=object)

In [76]:
ops.extend(basis)

[I,
 χ0,
 χ1,
 χ2,
 χ3]

In [63]:
for i in range(len(basis)):
    v = basis[i]
    us = basis[:i]
    u = v - (us.conjugate() * v) @ us
    u_norm = u.norm()
    if u_norm > 1.e-10:
        basis[i] = u / u_norm
    else:
        basis[i] = zero()
basis = basis[basis != 0]

In [70]:
basis

array([0.07 I -0.03 χ0 -0.57 χ1 +0.09 χ2 +0.81 χ3,
       0.32 I -0.53 χ0 -0.58 χ1 -0.33 χ2 -0.42 χ3,
       -0.14 I -0.38 χ0 +0.42 χ1 -0.72 χ2 +0.37 χ3,
       0.41 I -0.61 χ0 +0.38 χ1 +0.54 χ2 +0.14 χ3,
       0.84 I +0.44 χ0 +0.15 χ1 -0.27 χ2 +0.08 χ3], dtype=object)

In [72]:
(basis.conjugate().reshape(-1,1) @ basis.reshape(1,-1)).astype(numpy.double).round(10)

array([[ 1.,  0.,  0., -0., -0.],
       [ 0.,  1.,  0., -0., -0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [-0., -0.,  0.,  1.,  0.],
       [-0., -0.,  0.,  0.,  1.]])

In [83]:
isinstance((maj(i) for i in range(2)), Iterable)

True

In [81]:
list({1,2})

[1, 2]

In [87]:
outer = numpy.frompyfunc(lambda op1, op2: op1 @ op2, 2, 1).outer

In [89]:
outer(ops.basis, ops.basis)

array([[I, χ0, χ1, χ2, χ3],
       [χ0, I, χ0 χ1, χ0 χ2, χ0 χ3],
       [χ1, - χ0 χ1, I, χ1 χ2, χ1 χ3],
       [χ2, - χ0 χ2, - χ1 χ2, I, χ2 χ3],
       [χ3, - χ0 χ3, - χ1 χ3, - χ2 χ3, I]], dtype=object)

In [123]:
%run qboot

In [124]:
a = Operators([maj()]+[maj(i) for i in range(4)]+[maj(i,i+1) for i in range(3)])
a

Operators([I, χ0, χ1, χ2, χ3, χ0 χ1, χ1 χ2, χ2 χ3])

In [132]:
(0.01*a).round(2)

Operators([0.01 I, 0.01 χ0, 0.01 χ1, 0.01 χ2, 0.01 χ3, 0.01 χ0 χ1,
           0.01 χ1 χ2, 0.01 χ2 χ3])

In [139]:
numpy.tensordot(a, maj(1,2,3))

array([I, χ0, χ1, χ2, χ3, χ0 χ1, χ1 χ2, χ2 χ3, χ1 χ2 χ3], dtype=object)

In [140]:
a

Operators([I, χ0, χ1, χ2, χ3, χ0 χ1, χ1 χ2, χ2 χ3])

In [138]:
ops = Operators([[[maj(0)],[maj(0)]],[[maj(0)],[maj(0)]]])
for i, op in enumerate(ops.flat[0:2]):
    print(i, op)

0 χ0
1 χ0


In [102]:
numpy.array2string(numpy.array([maj(i) for i in range(4)]))

'[χ0 χ1 χ2 χ3]'

In [2]:
import numpy

In [44]:
class A():
    def __init__(self, real, imag):
        self._real = real
        self._imag = imag
        
    def __repr__(self):
        return repr((self._real, self._imag))
    
    @property
    def H(self):
        return self.conjugate()
    
    @property
    def real(self):
        return self._real
        
    def conjugate(self):
        return A(self._real, -self._imag)

In [45]:
A(1,2).real, A(1,2).conjugate(), A(1,2).H

(1, (1, -2), (1, -2))

In [11]:
numpy.real(A(1,2))

1

In [12]:
numpy.array([A(1,2)]).real

array([(1, 2)], dtype=object)

In [13]:
numpy.real(numpy.array([A(1,2)]))

array([(1, 2)], dtype=object)

In [14]:
numpy.array([A(1,2)]).conjugate()

array([(1, -2)], dtype=object)

In [15]:
numpy.array([A(1,2)]).H

AttributeError: 'numpy.ndarray' object has no attribute 'H'

In [67]:
class As(numpy.ndarray):
    def __new__(cls, input_array):
        obj = numpy.asarray(input_array).view(cls)
        return obj
    
    @property
    def real(self):
        return numpy.frompyfunc(lambda x: x.real,1,1)(self)
    
    @property
    def imag(self):
        return numpy.frompyfunc(lambda x: x.imag,1,1)(self)

In [68]:
As(A(1,2))

As((1, 2), dtype=object)

In [70]:
As([A(1,2)]).real, As([A(1,2)]).conjugate()

(As([1], dtype=object), As([(1, -2)], dtype=object))

In [66]:
numpy.dtype(numpy.float)

dtype('float64')

In [73]:
numpy.asarray([i for i in range(4)])

array([0, 1, 2, 3])

In [41]:
%run qboot

In [42]:
ops = OperatorSpace([maj(),maj(1)])
ops

OperatorSpace([I, χ1])

In [43]:
ops.represent(maj())

array([1., 0.])

In [44]:
ops == 1

OperatorSpace([ True, False])

In [45]:
ops != 1

OperatorSpace([False,  True])

In [46]:
ops.identity

I

In [47]:
ops.nonidentities

OperatorSpace([χ1])

In [59]:
%run qboot

In [110]:
us = Operators([maj(1),maj(2)])
us, us.shape

(Operators([χ1, χ2]), (2,))

In [111]:
us.H * maj(1)

array([1., 0.])

In [112]:
us.dot(us.H * maj(1)), (us.H * maj(1)).dot(us)

(χ1, χ1)

In [113]:
us @ (us.H * maj(1)), (us.H * maj(1)) @ us

(χ1, Operators(χ1))

In [114]:
numpy.tensordot(us, us.H * maj(1), axes=([0],[0]))

array(χ1, dtype=object)

In [105]:
us = Operators([maj(1),maj(2)])[:0]
us, us.shape

(Operators([]), (0,))

In [106]:
us.H * maj(1)

array([], dtype=float64)

In [107]:
us.dot(us.H * maj(1)), (us.H * maj(1)).dot(us)

(None, None)

In [108]:
us @ (us.H * maj(1)), (us.H * maj(1)) @ us

(0, Operators(0))

In [109]:
numpy.tensordot(us, us.H * maj(1), axes=([0],[0]))

array(None, dtype=object)

In [89]:
us = Operators([])
us, us.shape

(Operators([]), (0,))

In [90]:
us.H * maj(1)

array([], dtype=float64)

In [84]:
us.dot(us.H * maj(1)), (us.H * maj(1)).dot(us)

(0.0, 0.0)

In [85]:
us @ (us.H * maj(1)), (us.H * maj(1)) @ us

(0.0, Operators(0.))

In [86]:
numpy.tensordot(us, us.H * maj(1), axes=([0],[0]))

array(0.)

In [88]:
x = numpy.array([1,2,3])[:0]
x.dot(x)

0

In [93]:
us.size

0

In [13]:
class A():
    _a = 0
    def put(self, a):
        self._a = a
        
    def get(self):
        return self._a

x = A()
y = A()
x.put(1)
y.put(2)
x.get(), y.get()

(1, 2)

In [19]:
x=A()
x.__dict__

{}

In [35]:
%run qboot

In [36]:
x = maj(1,2)
y = maj(2,3)
z = x + y

In [37]:
x, y, z

(χ1 χ2, χ2 χ3, χ1 χ2 + χ2 χ3)

In [185]:
%run qboot

In [186]:
ops = numpy.random.randn(8,4)@Operators([maj(i) for i in range(4)])
ops

Operators([0.88 χ0 +0.38 χ1 +1.03 χ2 +1.41 χ3,
           -0.59 χ0 +0.23 χ1 -1.63 χ2 -1.80 χ3,
           1.67 χ0 -1.40 χ1 -1.18 χ2 -0.55 χ3,
           -0.16 χ0 +1.73 χ1 +1.46 χ2 -0.40 χ3,
           -0.73 χ0 -1.16 χ1 +0.56 χ2 +1.87 χ3,
           1.43 χ0 -0.06 χ1 -0.26 χ2 -1.49 χ3,
           0.45 χ0 -0.25 χ1 -1.45 χ2 -0.00 χ3,
           -0.65 χ0 -0.57 χ1 +2.50 χ2 +0.20 χ3])

In [187]:
ops = ops.orth()
ops

OperatorSpace([-0.27 χ0 +0.09 χ1 +0.79 χ2 +0.54 χ3,
               -0.19 χ0 +0.70 χ1 +0.29 χ2 -0.63 χ3,
               0.90 χ0 -0.06 χ1 +0.41 χ2 -0.14 χ3,
               0.29 χ0 +0.71 χ1 -0.34 χ2 +0.54 χ3])

In [191]:
ops.inner(maj(1))

array([ 0.0865134 +0.j,  0.69501108+0.j, -0.05610874+0.j,  0.71156647+0.j])

In [188]:
(ops.reshape(-1,1).inner(ops.reshape(1,-1))).round(10)

array([[ 1.+0.j,  0.+0.j,  0.+0.j, -0.+0.j],
       [ 0.+0.j,  1.+0.j, -0.+0.j, -0.+0.j],
       [ 0.+0.j, -0.+0.j,  1.+0.j, -0.+0.j],
       [-0.+0.j, -0.+0.j, -0.+0.j,  1.+0.j]])

In [189]:
ops.outer(ops)

Operators([[1.00 I,
            -0.17 χ0 χ1 +0.07 χ0 χ2 +0.27 χ0 χ3 -0.53 χ1 χ2 -0.43 χ1 χ3 -0.66 χ2 χ3,
            -0.06 χ0 χ1 -0.83 χ0 χ2 -0.44 χ0 χ3 +0.08 χ1 χ2 +0.02 χ1 χ3 -0.33 χ2 χ3,
            -0.22 χ0 χ1 -0.13 χ0 χ2 -0.30 χ0 χ3 -0.60 χ1 χ2 -0.33 χ1 χ3 +0.61 χ2 χ3],
           [0.17 χ0 χ1 -0.07 χ0 χ2 -0.27 χ0 χ3 +0.53 χ1 χ2 +0.43 χ1 χ3 +0.66 χ2 χ3,
            1.00 I,
            -0.61 χ0 χ1 -0.33 χ0 χ2 +0.60 χ0 χ3 +0.30 χ1 χ2 -0.13 χ1 χ3 +0.22 χ2 χ3,
            -0.33 χ0 χ1 -0.02 χ0 χ2 +0.08 χ0 χ3 -0.44 χ1 χ2 +0.83 χ1 χ3 -0.06 χ2 χ3],
           [0.06 χ0 χ1 +0.83 χ0 χ2 +0.44 χ0 χ3 -0.08 χ1 χ2 -0.02 χ1 χ3 +0.33 χ2 χ3,
            0.61 χ0 χ1 +0.33 χ0 χ2 -0.60 χ0 χ3 -0.30 χ1 χ2 +0.13 χ1 χ3 -0.22 χ2 χ3,
            1.00 I,
            0.66 χ0 χ1 -0.43 χ0 χ2 +0.53 χ0 χ3 -0.27 χ1 χ2 +0.07 χ1 χ3 +0.17 χ2 χ3],
           [0.22 χ0 χ1 +0.13 χ0 χ2 +0.30 χ0 χ3 +0.60 χ1 χ2 +0.33 χ1 χ3 -0.61 χ2 χ3,
            0.33 χ0 χ1 +0.02 χ0 χ2 -0.08 χ0 χ3 +0.44 χ1 χ2 -0.83 χ1 χ3 +0.06 χ2 χ3,
        

In [153]:
%load_ext snakeviz

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [171]:
%run qboot

In [172]:
from itertools import combinations
def c(a):
    return (maj(2*a) + 1j*maj(2*a+1))/2
basis = [maj()]
for n in range(1,3):
    for comb in combinations(range(4),n):
        op = opprod([c(i) for i in comb])
        basis.append(op.real)
        basis.append(op.imag)
base = OperatorSpace(basis).orth()
base.size

21

In [173]:
ope = base.outer(base)
ope.size **2

194481

In [183]:
base

OperatorSpace([I, χ2, χ3, χ4, χ5, χ6, χ7, χ0, χ1,
               0.71i χ2 χ6 -0.71i χ3 χ7, -0.71i χ4 χ7 -0.71i χ5 χ6,
               0.71i χ4 χ6 -0.71i χ5 χ7, 0.71i χ0 χ2 -0.71i χ1 χ3,
               -0.71i χ0 χ5 -0.71i χ1 χ4, 0.71i χ0 χ4 -0.71i χ1 χ5,
               -0.71i χ0 χ7 -0.71i χ1 χ6, 0.71i χ0 χ6 -0.71i χ1 χ7,
               -0.71i χ2 χ5 -0.71i χ3 χ4, 0.71i χ2 χ4 -0.71i χ3 χ5,
               -0.71i χ2 χ7 -0.71i χ3 χ6,
               -0.71i χ0 χ3 +(0.00-0.71i) χ1 χ2])

In [174]:
%snakeviz (ope.orth());

 
*** Profile stats marshalled to file '/var/folders/1m/3nz1kxmj2mgb2s2gwq2ndxqh0000gn/T/tmpzax5jese'. 
Embedding SnakeViz in this document...


In [146]:
ops = OperatorSpace([pauli(i) for i in range(4)])
ops

OperatorSpace([I, X0, Y0, Z0])

In [148]:
ope = ops.outer(ops)
ope

Operators([[I, X0, Y0, Z0],
           [X0, I, i Z0, -i Y0],
           [Y0, -i Z0, I, i X0],
           [Z0, i Y0, -i X0, I]])

In [156]:
As = ops.represent(ope, axis=0)
As

array([[[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]],

       [[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j],
        [0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
        [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]])

In [51]:
from itertools import combinations
list(combinations([1,2,3,4],6))

[]

In [1]:
%run 'qboot.py'

ModuleNotFoundError: No module named 'numpy'

In [236]:
def Krylov_simple(H, n):
    Ks = [one(H)]
    for k in range(n):
        K = H @ Ks[k]
        a = Ks[k].inner(K)
        K = K - a * Ks[k]
        if k > 0:
            b = Ks[k-1].inner(K)
            K = K - b * Ks[k-1]
        if k + 1 < n :
            norm = K.norm()
            if norm > 0:
                K = K / norm
                Ks.append(K)
            else:
                break
    return Ks

In [237]:
def c(i,s):
    return (maj(4*i+2*s) + 1j * maj(4*i+2*s+1))/2
def cd(i,s):
    return (maj(4*i+2*s) - 1j * maj(4*i+2*s+1))/2
def n(i,s):
    return cd(i,s) @ c(i,s)
H = sum(cd(0,s) @ c(1,s)+cd(1,s) @ c(0,s) for s in [0,1]) \
    + sum(n(i,0) @ n(i,1) for i in [0,1])
H

0.5 I +0.25i χ2 χ3 +0.25i χ0 χ1 -0.25 χ0 χ1 χ2 χ3 +0.25i χ6 χ7 +0.25i χ4 χ5 -0.25 χ4 χ5 χ6 χ7 +0.5i χ0 χ5 -0.5i χ1 χ4 +0.5i χ2 χ7 -0.5i χ3 χ6

In [238]:
Ks = Krylov_simple(H, 10)
Ks

[I,
 0.213i χ2 χ3 +0.213i χ0 χ1 -0.213 χ0 χ1 χ2 χ3 +0.213i χ6 χ7 +0.213i χ4 χ5 -0.213 χ4 χ5 χ6 χ7 +0.426i χ0 χ5 -0.426i χ1 χ4 +0.426i χ2 χ7 -0.426i χ3 χ6,
 0.065i χ2 χ3 +0.065i χ0 χ1 -0.065 χ0 χ1 χ2 χ3 +0.065i χ6 χ7 +0.065i χ4 χ5 -0.065 χ4 χ5 χ6 χ7 -0.0487i χ0 χ5 +0.0487i χ1 χ4 -0.0487i χ2 χ7 +0.0487i χ3 χ6 +0.268 χ2 χ3 χ6 χ7 -0.0893 χ2 χ3 χ4 χ5 -0.0893i χ2 χ3 χ4 χ5 χ6 χ7 -0.179 χ0 χ2 χ3 χ5 +0.179 χ1 χ2 χ3 χ4 -0.0893 χ0 χ1 χ6 χ7 +0.268 χ0 χ1 χ4 χ5 -0.0893i χ0 χ1 χ4 χ5 χ6 χ7 -0.179 χ0 χ1 χ2 χ7 +0.179 χ0 χ1 χ3 χ6 -0.0893i χ0 χ1 χ2 χ3 χ6 χ7 -0.0893i χ0 χ1 χ2 χ3 χ4 χ5 +0.0893 χ0 χ1 χ2 χ3 χ4 χ5 χ6 χ7 -0.179 χ0 χ5 χ6 χ7 +0.179 χ1 χ4 χ6 χ7 -0.179 χ2 χ4 χ5 χ7 +0.179 χ3 χ4 χ5 χ6 +0.357 χ0 χ2 χ5 χ7 -0.357 χ0 χ3 χ5 χ6 -0.357 χ1 χ2 χ4 χ7 +0.357 χ1 χ3 χ4 χ6,
 -0.0441i χ2 χ3 -0.0441i χ0 χ1 +0.129 χ0 χ1 χ2 χ3 -0.0441i χ6 χ7 -0.0441i χ4 χ5 +0.129 χ4 χ5 χ6 χ7 +0.0544i χ0 χ5 -0.0544i χ1 χ4 +0.0544i χ2 χ7 -0.0544i χ3 χ6 -0.0863 χ2 χ3 χ6 χ7 +0.0288 χ2 χ3 χ4 χ5 +0.114i χ2 χ3 χ4 χ5 χ6 χ7 -0.0276 χ0 χ2 χ3 χ5

In [239]:
%timeit Krylov_simple(H, 6)

9.27 ms ± 46.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Larger system

In [240]:
L = 3
H = sum(sum(cd(i,s) @ c((i+1)%L,s)+cd((i+1)%L,s) @ c(i,s) for s in [0,1]) for i in range(L)) \
    + sum(n(i,0) @ n(i,1) for i in range(L))
H

0.5i χ0 χ5 -0.5i χ1 χ4 +0.5i χ2 χ7 -0.5i χ3 χ6 +0.5i χ4 χ9 -0.5i χ5 χ8 +0.5i χ6 χ11 -0.5i χ7 χ10 -0.5i χ1 χ8 +0.5i χ0 χ9 -0.5i χ3 χ10 +0.5i χ2 χ11 +0.75 I +0.25i χ2 χ3 +0.25i χ0 χ1 -0.25 χ0 χ1 χ2 χ3 +0.25i χ6 χ7 +0.25i χ4 χ5 -0.25 χ4 χ5 χ6 χ7 +0.25i χ10 χ11 +0.25i χ8 χ9 -0.25 χ8 χ9 χ10 χ11

In [241]:
Ks = Krylov_simple(H, 10)
Ks

[I,
 0.265i χ0 χ5 -0.265i χ1 χ4 +0.265i χ2 χ7 -0.265i χ3 χ6 +0.265i χ4 χ9 -0.265i χ5 χ8 +0.265i χ6 χ11 -0.265i χ7 χ10 -0.265i χ1 χ8 +0.265i χ0 χ9 -0.265i χ3 χ10 +0.265i χ2 χ11 +0.132i χ2 χ3 +0.132i χ0 χ1 -0.132 χ0 χ1 χ2 χ3 +0.132i χ6 χ7 +0.132i χ4 χ5 -0.132 χ4 χ5 χ6 χ7 +0.132i χ10 χ11 +0.132i χ8 χ9 -0.132 χ8 χ9 χ10 χ11,
 0.1 χ0 χ1 χ4 χ5 +0.133 χ0 χ2 χ5 χ7 -0.133 χ0 χ3 χ5 χ6 +0.0667 χ0 χ4 χ5 χ9 -0.133 χ0 χ5 χ6 χ11 +0.133 χ0 χ5 χ7 χ10 -0.0667 χ0 χ1 χ5 χ8 -0.133 χ0 χ3 χ5 χ10 +0.133 χ0 χ2 χ5 χ11 -0.0667 χ0 χ2 χ3 χ5 -0.0667 χ0 χ5 χ6 χ7 -0.0667 χ0 χ5 χ10 χ11 +0.0667 χ0 χ5 χ8 χ9 -0.0667i χ0 χ5 χ8 χ9 χ10 χ11 -0.133 χ1 χ2 χ4 χ7 +0.133 χ1 χ3 χ4 χ6 -0.0667 χ1 χ4 χ5 χ8 +0.133 χ1 χ4 χ6 χ11 -0.133 χ1 χ4 χ7 χ10 +0.0667 χ0 χ1 χ4 χ9 +0.133 χ1 χ3 χ4 χ10 -0.133 χ1 χ2 χ4 χ11 +0.0667 χ1 χ2 χ3 χ4 +0.0667 χ1 χ4 χ6 χ7 +0.0667 χ1 χ4 χ10 χ11 -0.0667 χ1 χ4 χ8 χ9 +0.0667i χ1 χ4 χ8 χ9 χ10 χ11 +0.1 χ2 χ3 χ6 χ7 +0.133 χ2 χ4 χ7 χ9 -0.133 χ2 χ5 χ7 χ8 +0.0667 χ2 χ6 χ7 χ11 +0.133 χ1 χ2 χ7 χ8 -0.133 χ0 χ2 χ7 χ9 -0.0667 χ

In [243]:
%timeit Krylov_simple(H, 10)

835 ms ± 4.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [242]:
%prun Krylov_simple(H, 10)

         1815110 function calls in 1.246 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.255    0.025    1.230    0.123 qboot.py:133(__matmul__)
    83050    0.223    0.000    0.476    0.000 qboot.py:294(term_mul)
    83069    0.207    0.000    0.207    0.000 {method 'copy' of 'dict' objects}
    83069    0.133    0.000    0.387    0.000 qboot.py:104(__add__)
   158508    0.121    0.000    0.121    0.000 {built-in method numpy.array}
    79254    0.073    0.000    0.095    0.000 qboot.py:391(maj_term_mul)
    83078    0.061    0.000    0.097    0.000 qboot.py:89(__mul__)
   249227    0.035    0.000    0.035    0.000 qboot.py:49(__init__)
    79254    0.021    0.000    0.021    0.000 serialize.py:29(_numba_unpickle)
    83078    0.019    0.000    0.019    0.000 qboot.py:92(<dictcomp>)
    83069    0.019    0.000    0.116    0.000 qboot.py:94(__rmul__)
   328501    0.018    0.000    0.018    0.000 {built-in met